In [1]:
from __future__ import annotations

import json
import cProfile
import pstats

In [2]:
from py_research.db import (
    DataSource,
    RecMap,
    RelMap,
    SubMap,
    SelIndex
)

In [3]:
from test_db_schema import Search, Project, Task, User, Membership, Organization

In [4]:
data_source = DataSource(
    rec=Search,
    use_cache=False,
    push={
        "resultCount": Search.result_count,
        "search": Search.term,
        "results": RelMap(
            rel=Search.results,
            push={
                "project_name": Project.name,
                "project_start": Project.start,
                "project_end": Project.end,
                "project_status": Project.status,
                "tasks": RelMap(
                    rel=Project.tasks,
                    push={
                        "task_name": Task.name,
                        "task_assignees": RelMap(
                            rel=Task.assignees,
                            push=User.name,
                            match=User.name,
                        ),
                        "task_status": Task.status,
                    },
                ),
                "members": RelMap(
                    rel=Project.members,
                    push={User.name, User.age},
                    link=RecMap(
                        push={
                            Membership.role,
                        },
                    ),
                ),
            },
            pull={
                Project.org: SubMap(
                    push={
                        "organization_name": Organization.name,
                        "organization_address": Organization.address,
                        "organization_city": Organization.city,
                    },
                ),
                Project.number: SelIndex()
            },
        ),
    },
)

In [5]:
with open("./nested_data.json") as f:
    data = json.load(f)

In [6]:
with cProfile.Profile() as pr:
  rec = data_source.load(data)
  db = data_source.cache
  
  stats = pstats.Stats(pr)

In [7]:
rec

{'term': 'test', 'result_count': 3, 'results': [{'number': 0, 'name': 'baking cake', 'start': '2020-01-01', 'end': '2020-01-04', 'status': 'done', 'org': {'name': 'Bakery', 'address': 'Main Street 1', 'city': 'Bakerville', 'projects': <LoadStatus.unloaded: 1>, '_id': <LoadStatus.unloaded: 1>}, 'tasks': [{'name': 'task1', 'project': <LoadStatus.unloaded: 1>, 'assignees': [{'name': 'John', 'age': <LoadStatus.unloaded: 1>, 'tasks': <LoadStatus.unloaded: 1>, '_id': <LoadStatus.unloaded: 1>}], 'status': 'todo', '_id': <LoadStatus.unloaded: 1>}, {'name': 'task2', 'project': <LoadStatus.unloaded: 1>, 'assignees': [{'name': 'John', 'age': <LoadStatus.unloaded: 1>, 'tasks': <LoadStatus.unloaded: 1>, '_id': <LoadStatus.unloaded: 1>}], 'status': 'todo', '_id': <LoadStatus.unloaded: 1>}, {'name': 'task3', 'project': <LoadStatus.unloaded: 1>, 'assignees': [{'name': 'Jane', 'age': <LoadStatus.unloaded: 1>, 'tasks': <LoadStatus.unloaded: 1>, '_id': <LoadStatus.unloaded: 1>}], 'status': 'done', '_id':

In [8]:
stats.sort_stats("cumulative")
stats.print_stats("py_research")

         375098 function calls (324274 primitive calls) in 0.326 seconds

   Ordered by: cumulative time
   List reduced from 1274 to 66 due to restriction <'py_research'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     26/1    0.001    0.000    0.247    0.247 C:\Users\lworm\Repos\py-research\src\py_research\db\importing.py:370(_map_record)
      229    0.000    0.000    0.227    0.001 C:\Users\lworm\Repos\py-research\src\py_research\db\base.py:1868(__hash__)
26585/229    0.029    0.000    0.227    0.001 C:\Users\lworm\Repos\py-research\src\py_research\hashing.py:33(gen_int_hash)
7285/2677    0.002    0.000    0.221    0.000 C:\Users\lworm\Repos\py-research\src\py_research\hashing.py:63(<genexpr>)
6550/2448    0.002    0.000    0.220    0.000 C:\Users\lworm\Repos\py-research\src\py_research\hashing.py:27(_hash_sequence)
12528/4896    0.010    0.000    0.217    0.000 C:\Users\lworm\Repos\py-research\src\py_research\hashing.py:29(<lambda>)
       26    0.000